In [ ]:
## Montando drive para pegar as pastas
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Criando Hierarquia de Pastas

In [ ]:
## Procurando a pasta
!ls '/content/drive/MyDrive/Colab Notebooks/fruits'

apple  banana  cherry  chickoo	grapes	kiwi  mango  orange  strawberry


In [ ]:
## Organizando as pastas de treino e teste no drive
data_dir = '/content/drive/MyDrive/Colab Notebooks/fruits' # Dados originais

train_dir = '/content/drive/MyDrive/fruits-dataset/train' #
test_dir = '/content/drive/MyDrive/fruits-dataset/test' #

In [ ]:
## Criando arquivos
import os
import shutil
from pathlib import Path ## Achar caminhos e fazer listagens

## Criando os diretórios
Path(train_dir).mkdir(parents=True, exist_ok=True)
Path(test_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
!ls '/content/drive/MyDrive/fruits-dataset/train'

In [ ]:
classes = os.listdir(data_dir) ## Armazena os rótulos
for cls in classes:
    cls_train = os.path.join(train_dir, cls)
    cls_test  = os.path.join(test_dir, cls)

    Path(cls_train).mkdir(parents=True, exist_ok=True)
    Path(cls_test).mkdir(parents=True, exist_ok=True)

In [ ]:
!ls '/content/drive/MyDrive/fruits-dataset/train'

apple  banana  cherry  chickoo	grapes	kiwi  mango  orange  strawberry


In [ ]:
!ls '/content/drive/MyDrive/fruits-dataset/test'

apple  banana  cherry  chickoo	grapes	kiwi  mango  orange  strawberry


# Divisão dos Dados em Treino e Teste

In [ ]:
from posix import close
from sklearn.model_selection import train_test_split

for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    images = os.listdir(cls_dir) ## Listando imagens dentro do arquivo original

    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    for image in train_images:
        src = os.path.join(cls_dir, image)
        dst = os.path.join(train_dir, cls, image)
        shutil.copyfile(src, dst)

    for image in test_images:
        src = os.path.join(cls_dir, image)
        dst = os.path.join(test_dir, cls, image)
        shutil.copyfile(src, dst)

# Gerando PyTorch DataSet e DataLoader

Precisamos criar um transform

In [ ]:
from torchvision import transforms, datasets

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [ ]:
from torch.utils.data import DataLoader

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definindo uma CNN From Scratch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self) -> None:
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3,32, kernel_size=3, stride = 1, padding=1)
        self.conv2 = nn.Conv2d(32,64, kernel_size=3, stride = 1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding = 0)
        self.fc1 = nn.Linear(64*32*32, 512)
        self.fc2 = nn.Linear(512, len(classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64*32*32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

model_scratch = SimpleCNN().to('cuda')

# Definindo a Função de Custo e Otimizador

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_scratch.parameters(),lr=0.001)

# Efetuando o Treinamento

In [ ]:
def train_model(model, epochs):
    model.train()  # Entrando no modo de treino
    for epoch in range(epochs):
        running_loss = 0.0

        # Iterando sobre os batches de dados
        for x_batch, y_batch in train_loader:
            # Resetando os gradientes
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(x_batch.to('cuda'))

            # Calculando o erro
            loss = criterion(y_pred.to('cpu'), y_batch)

            # Backward pass (backpropagation)
            loss.backward()

            # Atualizando os parâmetros
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Treinando o modelo por 10000 épocas
train_model(model_scratch, 10)

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:769: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/10, Loss: 3.3326
Epoch 2/10, Loss: 1.9954
Epoch 3/10, Loss: 1.6672
Epoch 4/10, Loss: 1.3426
Epoch 5/10, Loss: 1.0152
Epoch 6/10, Loss: 0.7862
Epoch 7/10, Loss: 0.6042
Epoch 8/10, Loss: 0.3940
Epoch 9/10, Loss: 0.2881
Epoch 10/10, Loss: 0.2615


# Definindo um Modelo para Transfer Learning

In [ ]:
from torchvision import models

model_transfer = models.resnet18(pretrained=True)

In [ ]:
# Congelar os Pesos da resnet18
for param in model_transfer.parameters():
    param.requires_grad = False

In [ ]:
# Trocar última camada
num_ftrs = model_transfer.fc.in_features

# Substituir a camada de saída
model_transfer.fc = nn.Linear(num_ftrs, len(classes))

# Criando um Otimizador para a Transferência de Aprendizado

In [ ]:
optimizer = optim.Adam(model_transfer.fc.parameters(), lr=0.001)

In [ ]:
model_transfer = model_transfer.to('cuda')

In [ ]:
train_model(model_transfer, 10)

Epoch 1/10, Loss: 2.1251
Epoch 2/10, Loss: 1.5179
Epoch 3/10, Loss: 1.0968
Epoch 4/10, Loss: 0.8881
Epoch 5/10, Loss: 0.7206
Epoch 6/10, Loss: 0.6271
Epoch 7/10, Loss: 0.5206
Epoch 8/10, Loss: 0.4864
Epoch 9/10, Loss: 0.4441
Epoch 10/10, Loss: 0.3871


# Computando Acurácias dos Modelos

In [ ]:
from sklearn.metrics import accuracy_score
import torch

def calculate_accuracy(model, test_loader) :
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad() :
        for images, labels in test_loader :
            images = images.to('cuda')
            labels = labels.to('cuda')

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    return accuracy_score(y_true, y_pred)

In [ ]:
accuracy_score_scratch = calculate_accuracy(model_scratch, test_loader)
accuracy_score_transfer = calculate_accuracy(model_transfer, test_loader)

print(f"Accuracy Score Scratch: {accuracy_score_scratch}")
print(f"Accuracy Score Transfer: {accuracy_score_transfer}")

Accuracy Score Scratch: 0.5416666666666666
Accuracy Score Transfer: 0.8611111111111112


In [ ]:
instancia_de_teste = torch.rand(3, 3, 128, 128).to('cuda')

In [ ]:
torch.max(model_transfer(instancia_de_teste), 1)

torch.return_types.max(
values=tensor([0.2796, 0.4237, 0.6647], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([4, 4, 4], device='cuda:0'))

In [ ]:
classes[4]

'chickoo'